In [1]:
initial_path = 'peptide-QML'
# initial_path = '..'

In [2]:
import sys
sys.path.append(initial_path)

%load_ext autoreload
%autoreload 2
from my_code import helper_classes as c
from my_code import pytorch_model as m
from my_code import quantum_nodes as q

In [3]:
# name of the notebook
name_notebook = "sweep_4h_er_Xqm_v2"

In [4]:
# sweep = c.Sweep(
#     name_notebook, 
#     initial_path=initial_path,
#     description="4aa (8qubits) Try to find the best measurement and number of layers for the quantum block using Angle Encoding and embedding dim=2.",
#     repetitions=list(range(1, 4)),
#     quantum_n_layers=[1,3,5,10,20],
#     quantum_measurement=[1,'all'],
#     embedding_n_layers=[0,1,2,4,6],
# )

In [5]:
# sweep.save()

In [6]:
sweep = c.Sweep.load(file_name=name_notebook+"-2", day='0921', initial_path=initial_path)

In [7]:
for sweep_point in sweep.points_left:

    # print the point of the sweep
    sweep.print_sweep_status(sweep_point['idx'])

    # -- QUANTUM BLOCK -- #
    n_qubits = 8
    quantum_layer = q.circuit(
        n_qubits = n_qubits, 
        device = "default.qubit.torch",
        device_options = {'shots': None},
        embedding = q.parts.AngleEmbedding, 
        embedding_ansatz = q.parts.Ansatz_11,
        block_ansatz = q.parts.Ansatz_11,
        final_ansatz = q.parts.Ansatz_11, # TODO CHECK IF THIS IS THE BEST OPTION
        measurement = q.parts.Measurement('Z', sweep_point['quantum_measurement']),
        embedding_n_layers = sweep_point['embedding_n_layers'],
        different_inputs_per_layer = False,
        block_n_layers = sweep_point['quantum_n_layers'],
        # wrapper_qlayer = pw.QLayerEmpty,
    )

    # -- MODEL -- #                    
    class Model(m.Model):
        def __init__(self):
            super(Model, self).__init__()
            self.fc1 = m.nn.Embedding(num_embeddings=19, embedding_dim=2)
            self.fc2 = m.Reshape()
            self.fc3 = quantum_layer()     
            if sweep_point['quantum_measurement'] == 'all':     
                self.fc4 = m.nn.Linear(n_qubits, 1)
            else:
                self.fc4 = m.nn.Linear(1, 1)

        def forward(self, x):
            for i in range(1,5):
                x = getattr(self, 'fc{}'.format(i))(x)
            return x

    model = Model()

    # set quantum layer
    model.set_quantum_layer(quantum_layer)

    # set the name and path of the model
    model.set_name_and_path(name_notebook, initial_path)

    #set sweep point
    model.set_sweep_point(
        sweep_uuid = sweep.uuid,
        sweep_point = sweep_point,
        day=sweep.day
    )

    # -- TRAIN -- #
    model.optimize_params(
        #save info
        save_model_info = True,
        description = 'sweep with 4 amino acids and 4 qubits using Angle Encoding (repeating embedding) and classical Embedding',    
        #data
        data=c.Data.load(initial_path=initial_path, file_name='PE_SCORES_4_Numbers'),
        # train options
        optimizer = m.optim.Adam,
        optimizer_options = {'lr': 0.01},
        num_epochs = 10,                
        batch_size = 32,
        # early stopping options
        stop_training_options = {
            'lookback_epochs': 3, 
            'threshold_slope': 0.001,
            'threshold_std_dev': 0.2
        },
        # metadata
        metadata = {}
    )

    # -- VALIDATION and PLOTS -- #
    model.plot_losses(fig_size=(6,6))
    model.validate(
        pct=1,
        add_to_results=True,
        plot=True,
        print_items=False
    )

    # -- SAVE RESULTS -- #
    model.save_results()

    # -- ADD DATA GENERATED TO SWEEP -- #
    sweep.add_data(
        idx=sweep_point['idx'],
        model_uuid=model.uuid,
        best_loss_train=model.results.loss_epoch.best,
        best_loss_test=model.results.loss_validation_epoch.best,
        loss_validation=model.mean_loss_validation,
        n_parameters=model.n_parameters,
        time_train=model.results.time_epoch.last,
    )
    sweep.save(csv=False) #TODO: save csv=False automatically when sweep is already saved ion csv

    



 --- SWEEP POINT 1/150: (1, 1, 1, 0) ---
 --- parameters sweeping: ['repetitions', 'quantum_n_layers', 'quantum_measurement', 'embedding_n_layers'] 



Epoch [0/10], Loss epoch: 21.8247, Loss validation: 33.4402
	 Validation string, 	 i: 9; 	 prediction: 0.5531, 	 target: -0.6800, 	 loss: 1.5205
	 Validation string, 	 i: 22; 	 prediction: -0.7616, 	 target: 0.9700, 	 loss: 2.9984
	 Validation string, 	 i: 2; 	 prediction: -0.5178, 	 target: -0.6300, 	 loss: 0.0126
Epoch [1/10], Loss epoch: 18.5708, Loss validation: 22.7104, Time remaining: ~0.0h 2.0m 14s
	 Validation string, 	 i: 4; 	 prediction: -0.3809, 	 target: -3.9600, 	 loss: 12.8100
	 Validation string, 	 i: 48; 	 prediction: -1.5724, 	 target: -8.6300, 	 loss: 49.8090
	 Validation string, 	 i: 6; 	 prediction: -0.8887, 	 target: -0.1200, 	 loss: 0.5909
Epoch [2/10], Loss epoch: 15.8108, Loss validation: 20.2584, Time remaining: ~0.0h 1.0m 37s
